# INTRO

In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

# IMPORT CSV _ lista de doutores

In [ ]:
df = pd.read_csv('doutoresFeaturesParte_1.csv', encoding='UTF-8', nrows=500) #, dtype=str, nrows=1000)

df = df.fillna(' ')

df = df.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': str,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': str,
    'Publicacoes': float,
    'IdadeAcademica': str,
    'ListaFiliacoes': str,
    'UltimaFiliacao': str,
})

df_doutores = df.copy(deep=True)
df_doutores

# Nova Coluna com Resultado
df_doutores['HistoricoFiliacoesEmIESInternacional'] = ''
df_doutores['UltimaFiliacaoEmIES'] = ''

# IMPORT CSV _ IES Brasil

In [ ]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni_br = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_br = df_uni_br[['NOME_DA_IES', 'SIGLA']]
df_uni_br = df_uni_br.fillna('')
df_uni_br = df_uni_br.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_br

# IMPORT CSV _ IES Mundial

In [ ]:
# Lista de IES Mundial
# https://zenodo.org/records/13965926
# https://zenodo.org/records/13965926

df_uni_world = pd.read_csv('ror.csv', encoding='UTF-8')

df_uni_world = df_uni_world[['name', 'aliases', 'acronyms', 'country.country_code']]
df_uni_world = df_uni_world.rename(columns={
    'country.country_code': 'PAIS',
    'name': 'NOME_DA_IES',
    'acronyms': 'SIGLA',
    'aliases': 'ALIASES'
})

# IES BR foram removidos pois os nomes não estão em inglês e assim ficaria duplicado em relação a dataset do MEC
df_uni_world = df_uni_world[df_uni_world['PAIS'] != 'BR']

df_uni_world = df_uni_world.fillna('')

df_uni_world

# PRE-PROCESSAMENTO

In [ ]:
def clean_names(df, column_name, remove_semicolon=True):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace e ;
    if remove_semicolon:
        df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)
    else:
        df[column_name] = df[column_name].str.replace('[^\w\s;]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [ ]:
df_doutores = df_doutores.apply(lambda x:x.astype(str).str.upper())
df_doutores = clean_names(df_doutores, 'ListaFiliacoes', False)
df_doutores = clean_names(df_doutores, 'UltimaFiliacao', False)

df_uni_br = df_uni_br.apply(lambda x:x.astype(str).str.upper())
df_uni_br = clean_names(df_uni_br, 'NOME_DA_IES')

In [ ]:
# Split lista de IES por inicio do nome
df_uni_br.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_br.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

# Merge IES df

In [ ]:
df_uni_br['PAIS'] = 'BR'
df_uni_br['ALIASES'] = ''

print( (df_uni_world.shape[0] + df_uni_br.shape[0]), ' = ', df_uni_world.shape[0], ' + ', df_uni_br.shape[0])

df_uni = pd.concat([df_uni_br, df_uni_world])
df_uni

# PLN

## TF_IDF

In [ ]:
import re

def get_uni_list_df(nome_instituicao):
    if 'INSTITUTO' in nome_instituicao:
        return df_uni_instituto, tfidf_matrix_instituto
    elif 'FACULDADE' in nome_instituicao:
        return df_uni_faculdades, tfidf_matrix_faculdades
    elif 'ESCOLA' in nome_instituicao:
        return df_uni_escola, tfidf_matrix_escola
    elif 'UNIVERSIDADE' in nome_instituicao:
        return df_uni_universidade, tfidf_matrix_universidade
    elif 'INSTITUICAO' in nome_instituicao:
        return df_uni_instituicao, tfidf_matrix_instituicao

    return df_uni_outros, tfidf_matrix_outros


def iterate_sigla(nome_instituicao):

    lista = df_uni["SIGLA"].apply(lambda x: True if x in nome_instituicao else False)
    siglas_encontradas = df_uni[lista]

    if siglas_encontradas.empty:
        return ''
    else:
        return siglas_encontradas['SIGLA'][0], siglas_encontradas['PAIS'][0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF : Lista IES Brasil
vectorizer_b = TfidfVectorizer().fit(df_uni_br['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer_b.transform(df_uni_br['NOME_DA_IES']) # geral
tfidf_matrix_instituto = vectorizer_b.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer_b.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer_b.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer_b.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer_b.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer_b.transform(df_uni_outros['NOME_DA_IES'])

# Vetorização TF-IDF : Lista IES Mundial
vectorizer_w = TfidfVectorizer().fit(df_uni_world['NOME_DA_IES'])
tfidf_matrix_world = vectorizer_w.transform(df_uni_world['NOME_DA_IES'])

## UltimaFiliacaoEmIES

In [ ]:
threshold = 0.8

def metodo1(nomes_instituicoes):

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, _ = iterate_sigla(nome_instituicao)
            if sigla_detectado:
                print("sigla detectado : ", sigla_detectado)
                print("==================")
                return '1'

            # Se nao contem a sigla, procurar por nome completo
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)
                max_index = np.argmax(similaridade)

                nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                if (max_value >= threshold):
                    print("nome detectado : ", nome_detectado, max_value)
                    print("==================")
                    return '1'

                # se nao achou nenhuma ies br, procurar ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)
                    max_index = np.argmax(similaridade)

                    nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                    if (max_value >= threshold):
                        print("nome detectado : ", nome_detectado, max_value)
                        print("==================")
                        return '1'

                    else:
                        print("==================")

                    # se nao passar no threshold, so lamento

        print("==================")
    return '0'

df_doutores['UltimaFiliacaoEmIES'] = df_doutores["UltimaFiliacao"].apply(lambda x: metodo1(x))

## HistoricoFiliacoesEmIESInternacional

In [ ]:
def metodo1(nomes_instituicoes):

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, pais = iterate_sigla(nome_instituicao)
            if pais != 'BR':
                print("sigla estrangeira detectado : ", sigla_detectado)
                print("==================")
                return '1'

            # Se nao contem a sigla, procurar por nome completo
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)
                max_index = np.argmax(similaridade)

                nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                if (max_value >= threshold):
                    print("nome detectado : ", nome_detectado, max_value)
                    print("==================")

                # se nao achou nenhuma ies br, procurar ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)
                    max_index = np.argmax(similaridade)

                    nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                    if (max_value >= threshold):
                        print("nome estrangeiro detectado : ", nome_detectado, max_value)
                        print("==================")
                        return '1'

                    else:
                        print("==================")

                    # se nao passar no threshold, so lamento

        print("==================")
    return '0'

df_doutores['HistoricoFiliacoesEmIESInternacional'] = df_doutores["ListaFiliacoes"].apply(lambda x: metodo1(x))

In [ ]:
df_doutores

# Export


In [ ]:
'''
df = df.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': str,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': str,
    'Publicacoes': float,
    'IdadeAcademica': str,
    'ListaFiliacoes': str,
    'UltimaFiliacao': str,
    'HistoricoFiliacoesEmIESInternacional': int,
    'UltimaFiliacaoEmIES': int,
})

df_doutores.to_csv('doutores_processado.csv')
'''

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_doutores.isna().sum()